In [1]:
import numpy as np
import pandas as pd

In [54]:
N = 100000
ts_data = np.random.rand(N)
ts_data = pd.DataFrame(ts_data)
v_col = ts_data.columns[0]
size = len(ts_data)

In [55]:
neighbor_data = pd.DataFrame(np.random.rand(N,10))

In [56]:
const_rel_dw = 0.1
const_rel_up = 0.1
const_abs = 0.1
value_min = -np.inf
value_max = np.inf

In [57]:
def find_invalid(ts_data, size, const_rel_dw, const_rel_up, neighbor_data, value_min, value_max) : 

    aux_matrix = np.zeros((size, 3))
    aux_matrix[:, 0] = neighbor_data.mean(axis=1).abs() * const_rel_dw
    aux_matrix[:, 1] = neighbor_data.mean(axis=1).abs() * const_rel_up
    aux_matrix[:, 2] = [const_abs] * size
    aux_matrix[:, 0] = np.nanmax(aux_matrix[:, [0, 2]], axis=1)
    aux_matrix[:, 1] = np.nanmax(aux_matrix[:, [1, 2]], axis=1)
    aux_matrix[:, 2] = neighbor_data.std(axis=1) * 2
    aux_matrix[:, 0] = np.nanmax(aux_matrix[:, [0, 2]], axis=1)
    aux_matrix[:, 1] = np.nanmax(aux_matrix[:, [1, 2]], axis=1)

    ts_data = ts_data.assign(
                min=neighbor_data.min(axis=1) - aux_matrix[:, 0],
                max=neighbor_data.max(axis=1) + aux_matrix[:, 1],
    )
    event_time_index = ts_data[
        ((value_min <= ts_data[v_col]) & (ts_data[v_col] <= value_max))
        & ((ts_data[v_col] < ts_data["min"]) | (ts_data[v_col] > ts_data["max"]))

    ].index
    
    return event_time_index

In [58]:
%%timeit
find_invalid(ts_data, size, const_rel_dw, const_rel_up, neighbor_data, value_min, value_max)

27.2 ms ± 498 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
